In [1]:
import json
from tqdm import tqdm
import time
import requests
from elasticsearch import Elasticsearch
from langchain import PromptTemplate
from langchain_core.prompts.few_shot import FewShotPromptTemplate


In [2]:
es_client=Elasticsearch("http://localhost:9200")

## Q1. Running Elastic

In [9]:
(requests.get("http://localhost:9200").json())['version']['build_hash']

'42f05b9372a9a4a470db3b52817899b99a76ee73'

In [4]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [6]:
index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
for doc in tqdm(documents):
    es_client.index(index=index_name,document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

100%|██████████| 948/948 [00:23<00:00, 41.16it/s]


In [8]:
query="How do I execute a command in a running docker container?"

In [12]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
            }
        }
    }

In [19]:
response=es_client.search(index=index_name,body=search_query,)

## Q2. Indexing the data

In [23]:
response['hits']['hits'][0]['_score']

84.050095

In [37]:
search_query_new = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

In [38]:
response_new=es_client.search(index=index_name,body=search_query_new)

In [39]:
##Filtering

In [40]:
response_new['hits']['hits'][2]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

In [43]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [46]:
records=[rec['_source'] for rec in response_new['hits']['hits']]

In [48]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [50]:
prompt = PromptTemplate.from_template(template=context_template)


In [53]:
records

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [70]:
example_prompt = PromptTemplate(
    input_variables=["question","text"],
    template=context_template
)

In [104]:
prefix="""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {query}

CONTEXT:
"""

In [105]:
query="How do I execute a command in a running docker container?"

In [106]:
suffix = """"""

In [107]:
few_shot_prompt_template = FewShotPromptTemplate(
    examples=records,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

In [108]:
print(len(few_shot_prompt_template.format(query=query)))

1462


In [109]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [110]:
len(encoding.encode((few_shot_prompt_template.format(query=query))))

323